## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from transformers import AutoTokenizer, AutoModel, BertModel
from torch import nn
import torch
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
import torch.nn.functional as F

from sklearn.preprocessing import QuantileTransformer

In [3]:
# Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.compose import ColumnTransformer

from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    f1_score,
)
from sklearn.metrics import balanced_accuracy_score

import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [5]:
import spacy, re

nlp = spacy.load("en_core_web_sm")

def remove_mentions_and_hashtags(text):
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    return text


def remove_numbers(text):
    text = re.sub(r"\d+", "", text)
    return text


def remove_punctuation(text):

    translator = str.maketrans("", "", string.punctuation)

    # Remove punctuation using the translation table
    text_without_punct = text.translate(translator)

    return text_without_punct


def remove_stopwords(text):
    filtered_sentence = []
    doc = nlp(text)
    for token in doc:
        if token.is_stop == False:
            filtered_sentence.append(token.text)
    return " ".join(filtered_sentence)


def clean_text(text):

    # Standardize text
    # text = standardize_accented_chars(text)

    # Remove URLs
    text = re.sub(r"http\S+", "", text)

    # Remove mentions and hashtags
    text = remove_mentions_and_hashtags(text)

    # Lowercase
    text = text.lower()

    # Remove punctuation
    text = remove_punctuation(text)

    # Remove numbers
    text = remove_numbers(text)

    # Remove all the special characters
    text = re.sub(r"\W", " ", text)

    # Remove stopwords
    text = remove_stopwords(text)

    # Substituting multiple spaces with single space
    text = re.sub(r"\s+", " ", text, flags=re.I)

    return text


def lemmatize(text):
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    return text

## Utils functions

In [25]:
def print_metrics(y_pred, y_test, title: str = "Confusion Matrix"):
    from sklearn.metrics import matthews_corrcoef

    print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred)}")
    print(f"Precision: {precision_score(y_test, y_pred,average='weighted')}")
    print(f"Recall: {recall_score(y_test, y_pred,average='weighted')}")
    print(f"F1: {f1_score(y_test, y_pred,average='weighted')}")
    print(f"Metthew corr: {matthews_corrcoef(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))

    # cm = confusion_matrix(y_test, y_pred)
    # sns.heatmap(cm, annot=True, fmt="g")
    # plt.title(title)
    # plt.xlabel("Predicted")
    # plt.ylabel("True")
    # plt.show()

In [36]:
def split_dataset(data):
    """Function to split dataset into train, val and test"""
    np.random.seed(112)
    df_train, df_val, df_test = np.split(
        data.sample(frac=1, random_state=42),
        [int(0.8 * len(data)), int(0.9 * len(data))],
    )

    # Print dimensions
    print(len(df_train), len(df_val), len(df_test))

    return df_train, df_val, df_test

## Traditional algorithms

### Data


In [14]:
current_directory = os.getcwd()

parent_directory = os.path.dirname(current_directory)

DATA_PATH = os.path.join(parent_directory,"data")
DATA_TOURNAMENT = os.path.join(DATA_PATH,"tournament_hints_data.parquet")

print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

Current Directory: /Users/marianluca/Projects/HackingBigNumbers/Tournament/marian
Parent Directory: /Users/marianluca/Projects/HackingBigNumbers/Tournament


In [16]:
df = pd.read_parquet(DATA_TOURNAMENT)
df

,commercial_name,business_tags,short_description,description,main_business_category
0,White Horse,Tile Manufacturing | European Aesthetics Ceram...,White Horse is highly regarded as a tile trail...,White Horse Ceramic Singapore is a leading man...,Tile Store
1,Wealth Solution Partners,Super and SMSF Services | Financial Planning a...,"WSP, Wealth Solution Partners, Financial Plann...",Wealth Solution Partners Pty Ltd is an indepen...,Investment Consultants & Financial Advisors
2,PMG,Fire and Water Cleanup Services | Mold Remedia...,PMG General Solutions Inc. is an environmental...,PMG General Solutions Inc. is an environmental...,Damage Restoration & Mold Remediation
3,TMP Capital PLLC,Licensed in AL & FL | 203K Loans | 15-year Fix...,TMP Capital PLLC Consulting Company Franklin M...,"TMP Capital PLLC Consulting Company, also know...",Mortgage Brokers
4,Genertek Power,Industrial and Commercial Energy Storage | Ass...,Genertek Power Ltd a UK electricity systems & ...,Genertek Power Limited is a privately-owned UK...,Renewable energy companies
...,...,...,...,...,...
626241,Global Golf Tech Solutions,Manufacturing | Golf Academy | G Launch Monito...,The best personal golf launch monitor screen p...,Global Golf Tech Solutions is a company that s...,Golf Courses & Country Clubs
626242,Renko,Latest Processing Technologies | E Gaskets for...,EPDM Rubber products from Renko can be found o...,RENKO is a company that has been producing hig...,Fabricated Rubber Products
626243,Norstal,Residential Buildings | Custom Project Service...,Norstal produces a broad range of steel struct...,Norstal is a steel structure producer that spe...,Metal Fabrication Services
626244,Acoustic,Wood-based Acoustic Products Manufacturer | De...,"We are designed and manufactured in UAE, Acous...",Acoustic.ae is a member of a UAE-based group o...,Building Material Manufacturers


In [23]:
columns_round = set(df.columns)
columns_round.remove("main_business_category")
print(columns_round)

{'short_description', 'commercial_name', 'business_tags', 'description'}


In [31]:
from sklearn.preprocessing import LabelEncoder

# Creating an instance of LabelEncoder
encoder = LabelEncoder()

# Fitting the encoder
encoder.fit(df["main_business_category"].unique())

# Transforming the data
df["target"] = encoder.transform(df["main_business_category"])
# print("Encoded data:", encoded_data)

# Inverse transforming the data
# decoded_data = encoder.inverse_transform(encoded_data)
# print("Decoded data:", decoded_data)

In [32]:
df["target"]

0         521
1         286
2         145
3         348
4         461
         ... 
626241    239
626242    197
626243    333
626244     79
626245    390
Name: target, Length: 626246, dtype: int64

In [37]:
df_train, df_val, df_test = split_dataset(df)

500996 62625 62625


/Users/marianluca/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


### Algorithms


In [41]:
def apply_bow(X_train, X_test):
    # count_vect = CountVectorizer()

    from sklearn.feature_extraction.text import TfidfVectorizer

    tfidf_transformer = TfidfVectorizer()

    # X_train = count_vect.fit_transform(X_train)
    X_train = tfidf_transformer.fit_transform(X_train)

    # X_test = count_vect.transform(X_test)
    X_test = tfidf_transformer.transform(X_test)

    return X_train, X_test


def apply_traditional_algorithms(df_train, df_test, pipeline_cls, threshold=0.5):

    X_train, y_train = df_train["clean_text"], df_train["target"]
    X_test, y_test = df_test["clean_text"], df_test["target"]

    X_train, X_test = apply_bow(X_train, X_test)

    best_model, best_model_name = None, None
    base_f1_score = 0
    for Name, cls in pipeline_cls.items():
        cls.fit(X_train, y_train)
        y_proba = cls.predict_proba(X_test)
        y_pred = (y_proba[:, 1] >= threshold).astype(int)

        print(Name)
        print(cls)
        print("Metrics for " + str(Name) + "\n")
        print(classification_report(y_test, y_pred))

        print_metrics(y_pred, y_test, Name + " - Text")

        if f1_score(y_test, y_pred, average="macro") > base_f1_score:
            base_f1_score = f1_score(y_test, y_pred, average="macro")
            best_model = cls
            best_model_name = Name

        print(f"Test set F1-score: {f1_score(y_test, y_pred, average='macro')}")
        print(f"Test set Precision {precision_score(y_test,y_pred,average='macro')}")
        print(f"Test set Recall {recall_score(y_test, y_pred,average='macro')}")
        print("\n\n")
        print(
            "-----------------------------------------------------------------------------------------------------------------------"
        )

    return best_model, best_model_name, base_f1_score

In [42]:
pipeline_classifiers = {
    "Logistic Regression": LogisticRegression(),
    "RandomForestClassifier": RandomForestClassifier(class_weight="balanced"),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "SVC": SVC(class_weight="balanced", probability=True),
}

In [54]:
for col_name in columns_round:
    df_tmp_train, df_tmp_test = (
        df_train[[col_name, "target"]],
        df_test[[col_name, "target"]],
    )

    df_tmp_train.rename(columns={col_name: "clean_text"}, inplace=True)
    df_tmp_test.rename(columns={col_name:"clean_text"},inplace=True)

    apply_traditional_algorithms(df_tmp_train, df_tmp_test, pipeline_classifiers)

/var/folders/kc/3h9bj6ps6t965wjwwkvd77nh0000gn/T/ipykernel_6597/995227637.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp_train.rename(columns={col_name: "text"}, inplace=True)
/var/folders/kc/3h9bj6ps6t965wjwwkvd77nh0000gn/T/ipykernel_6597/995227637.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp_test.rename(columns={col_name:"text"},inplace=True)


KeyError: 'clean_text'

In [46]:
df_tmp = df_train

In [51]:
df_tmp.rename(columns={"commercial_name":"ok"},inplace=True)

In [52]:
df_tmp

,ok,business_tags,short_description,description,main_business_category,target
152188,DFE,Drilling Fluids Manufacturer | Dewatering Syst...,DFE can provide experienced personnel for soli...,DFE New Zealand Limited is a local company tha...,Oil & Gas - Extraction & Distribution,372
41010,Intelliblock Technologies,Initial Coin Offering | ESG Solutions | Privat...,IntelliBlock Technologies is a leading blockch...,IntelliBlock Technologies is a blockchain-base...,Cryptocurrency,138
181456,Blockstart Ventures,Reducing Risk | Enterprise Blockchain Investme...,Blockstart Ventures LP is multi-strategy inves...,Blockstart Ventures LP is a multi-strategy inv...,Investment Firms & Venture Capital,287
268221,Old Strathcona Performing Arts Centre,Performing Arts Venue | Event Venue Rental Ser...,The Old Strathcona Performing Arts Centre is a...,The Old Strathcona Performing Arts Centre (Osp...,Event Planner Services,192
289856,Sylvan Transportation,Airport Transportation Services | Family-frien...,Whether you are needing travel assistance to t...,Sylvan Transportation is a transportation comp...,"Buses, Shuttles & Local Transit",80
...,...,...,...,...,...,...
497826,PRTI,Consultancy Services | FDP / SDP VTU Internshi...,"PRTI is a learning, tutoring & courses provide...",Punitharaj Research and Training Institute - B...,"Learning, Tutoring & Courses",299
600087,Accel Computers,Notebooks | Networking Solutions | Voice Over ...,"ACCEL COMPUTERS – Full Technology, Networking,...",Accel Computers is a computer technology compa...,Computer Repair & Service,124
175758,NuWatt,Switches and Thermometers | Pressure Relief Va...,"NuWatt Technologies, Leading Supplier Of Elect...",NuWatt Technologies is a South African company...,Electric Supplies & Power Generation,173
442220,Oakdale Child And Family Service,Non-profit Organization | Residential Services...,Oakdale Child & Family Service Inc. provides r...,Oakdale Child and Family Service Inc. has been...,Senior Care and Assisted Living Facilities,478


## Bert

In [ ]:
import torch.nn as nn
from transformers import BertTokenizer, BertModel


class BERTClass(nn.Module):

    def __init__(
        self,
        bert_model: str = "bert-base-uncased",
        num_classes: int = 2,
        droput_rate: int = 0.3,
    ):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained(bert_model, return_dict=True)
        self.dropout = nn.Dropout(droput_rate)
        self.linear = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)

        return output